In [9]:
from transformers import Wav2Vec2ForCTC, AutoProcessor

finetune_model_id = "./models/wav2vec2-large-mms-1b-shan"
original_model_id = "facebook/mms-1b-all"

finetune_model = Wav2Vec2ForCTC.from_pretrained(finetune_model_id, target_lang="shn", ignore_mismatched_sizes=True).to("cuda")
finetune_processor = AutoProcessor.from_pretrained(finetune_model_id)

original_model = Wav2Vec2ForCTC.from_pretrained(original_model_id).to("cuda")
original_processor = AutoProcessor.from_pretrained(original_model_id)
original_processor.tokenizer.set_target_lang("shn")
original_model.load_adapter("shn")


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Some weights of the model checkpoint at facebook/mms-1b-all were not used when initializing Wav2Vec2ForCTC: ['wav2vec2.encoder.pos_conv_embed.conv.weight_g', 'wav2vec2.encoder.pos_conv_embed.conv.weight_v']
- This IS expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/mms-1b-all and are newly initialized: ['wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original0', 'wav2vec2

In [3]:
from datasets import Audio, load_dataset

test_data = load_dataset("NorHsangPha/shn-asr-datasets", split="test", token=True)
test_data = test_data.cast_column("audio", Audio(sampling_rate=16_000))

In [6]:
print(test_data[20]["audio"])
print(test_data[20]["transcription"])

{'path': 'audio35.wav', 'array': array([-0.0038032 , -0.00612239, -0.00519261, ..., -0.00181367,
       -0.00168416, -0.00077092]), 'sampling_rate': 16000}
သိူဝ်ႈႁတ်ႉၶႅၼ်ပွတ်းသီၶဵဝ်လိူင် ထႅင်ႈပၼ်တၢင်းႁၢင်ႈလီၸွမ်းၽိဝ်ၼိူဝ်ႉၼင်ၶၢဝ်လိူင်ဢွၼ်ႇမၼ်းၼၢင်းၼၼ်ႉထူၼ်ႈ


In [7]:
selected_data_sample = test_data[20]
audio_samples = selected_data_sample["audio"]["array"]

In [37]:
import librosa
import pandas as pd

ASR_SAMPLING_RATE = 16_000
sample_num = 301
audio_fp = f"../../Datasets/dataset_2/audio-data/train/audio{sample_num}.wav"
df = pd.read_csv("../../Datasets/dataset_2/metadata.csv")


audio_samples = librosa.load(audio_fp, sr=ASR_SAMPLING_RATE, mono=True)[0]

In [38]:
import torch


finetune_input_dict = finetune_processor(audio_samples, sampling_rate=16_000, return_tensors="pt")
finetune_logits = finetune_model(finetune_input_dict.input_values.to("cuda")).logits
finetune_pred_ids = torch.argmax(finetune_logits, dim=-1)[0]

original_input_dict = original_processor(audio_samples, sampling_rate=16_000, return_tensors="pt")
original_logits = original_model(original_input_dict.input_values.to("cuda")).logits
original_pred_ids = torch.argmax(original_logits, dim=-1)[0]

c:\Users\NoerN\.conda\envs\mmslabs\lib\site-packages\torch\nn\modules\conv.py:306: UserWarning: Plan failed with a cudnnException: CUDNN_BACKEND_EXECUTION_PLAN_DESCRIPTOR: cudnnFinalize Descriptor Failed cudnn_status: CUDNN_STATUS_NOT_SUPPORTED (Triggered internally at ..\aten\src\ATen\native\cudnn\Conv_v8.cpp:919.)
  return F.conv1d(input, weight, bias, self.stride,


In [39]:
from IPython.display import Markdown, display

def printmd(string, color=None):
    colorstr = "<span style='color:{}'>{}</span>".format(color, string)
    display(Markdown(colorstr))


printmd("\nFinetune model Prediction:", color="orange")
print(finetune_processor.decode(finetune_pred_ids))

printmd("\nOriginal model Prediction:", color="yellow")
print(original_processor.decode(original_pred_ids))

printmd("\nReference:", color="green")
# print(selected_data_sample["transcription"])
print(df["transcription"][sample_num-1])

<span style='color:orange'>
Finetune model Prediction:</span>

ၼႂ်းၶၢ်းတၢင်းသိပ်းပီဢၼ်ဢမ်ႇလႆႈႁူပ်ႉၺႃးၵၼ်ၼၼ်ႉၵေႃႈ မၼ်းပဵၼ်လွင်ႈတၢင်ႈၸႂ်ႁဝ်းမိူၼ်ၵၼ်


<span style='color:yellow'>
Original model Prediction:</span>

ၼႂ်ႉ ၶႃး တၢင်း သိပ်ႉ ပီ ဢၼ် ဢမ်ႇ လႆႈ ႁွပ်ႉၺႃး ၵၼ် ၼၼ်ႉ ၵေႃႈ မၼ်း ပိူၼ်ႇ လွင်ႈ တၢင်ႈၸႂ် ႁဝ်း မိူၼ်ႇ ၵၼ် -


<span style='color:green'>
Reference:</span>

ၼႂ်းၶၢဝ်းတၢင်းသိပ်းပီ ဢၼ်ဢမ်ႇလႆႈႁူပ်ႉၺႃးၵၼ်လႆႈၼၼ်ႉၵေႃႈ မၼ်းပဵၼ်လွင်ႈတင်ႈၸႂ်ႁဝ်းမိူၼ်ၵၼ်


In [ ]:
from transformers import Wav2Vec2CTCTokenizer

tokenizer = Wav2Vec2CTCTokenizer.from_pretrained(finetune_model_id)

token = tokenizer("ၾႃႉၾူၼ်ၵမ်ႇလမ်သႃး")

tokenizer.decode(token["input_ids"])

In [ ]:
finetune_model.push_to_hub("haohaa/wav2vec2-large-mms-1b-shan")